# DataFrame Creating

Из каждого html файла берем необходимую информацию и добавляем в новый дата фрейм.

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import re
import csv
import numpy as np

In [5]:
pages = os.listdir(os.getcwd() + "/pages")

In [3]:
pages

['152919717.html',
 '164987461.html',
 '197214440.html',
 '231550821.html',
 '242690916.html',
 '242691237.html',
 '244613901.html',
 '246138953.html',
 '257998425.html',
 '258448511.html',
 '261785479.html',
 '263220642.html',
 '266889143.html',
 '266942924.html',
 '267686213.html',
 '268959829.html',
 '269546998.html',
 '269549003.html',
 '270236085.html',
 '270410257.html',
 '271613372.html',
 '271620750.html',
 '271715524.html',
 '271865863.html',
 '271925324.html',
 '272284062.html',
 '272383012.html',
 '272498239.html',
 '272557400.html',
 '272629978.html',
 '273380996.html',
 '273484435.html',
 '273536472.html',
 '273852300.html',
 '275282175.html',
 '275404384.html',
 '275801274.html',
 '276028964.html',
 '276109526.html',
 '276134091.html',
 '276354185.html',
 '276489793.html',
 '277311730.html',
 '277370296.html',
 '277524435.html',
 '277528794.html',
 '277965361.html',
 '278557244.html',
 '278657123.html',
 '278849521.html',
 '279027238.html',
 '279260458.html',
 '279416943.

In [8]:
def get_page_data(files: str):
    """Проходится по всем Html ссылкам в папке и парсит необходимую информацию в dataframe."""
    lst = list()
    for file in files:
        data = dict()
        with open(os.getcwd() + "/pages/" + file, "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "lxml")

            try:
                data["Название"] = soup.find(
                    "h1", class_=re.compile("\.*--title--\.*")
                ).text
            except:
                data["Название"] = np.nan
            try:
                addr = soup.find_all("a", class_=(
                    "a10a3f92e9--address--SMU25"))
                address = ""
                for part in addr:
                    if part != addr[-1]:
                        address += part.text + ", "
                    else:
                        address += part.text
                data["Адрес"] = address
            except:
                data["Адрес"] = np.nan
            try:
                data["Метро"] = soup.find(
                    "a", class_=re.compile("\.*--underground_link--\.*")
                ).text
            except:
                data["Метро"] = np.nan
            try:
                data["Время до метро"] = soup.find(
                    "span", class_=re.compile("\.*--underground_time--\.*")
                ).text
            except:
                data["Время до метро"] = np.nan
            try:
                data["Цена"] = (
                    soup.find("div", class_=("a10a3f92e9--amount--ON6i1"))
                    .text[:-1]
                    .replace("\xa0", "")
                )
            except:
                data["Цена"] = np.nan
            try:
                price_div = soup.find("div", class_="a10a3f92e9--item--iWTsg")
                data["Цена за квадрат"] = (
                    price_div.find_all("span")[-1].text[:-5].replace(" ", "")
                )
            except:
                data["Цена за квадрат"] = np.nan
            try:
                info_flat = soup.find_all(
                    "div", class_=("a10a3f92e9--item--Jp5Qv"))
                for span in info_flat:
                    name = span.find(
                        "span", class_=re.compile("\.*--color_gray60_100--\.*")
                    ).text
                    value = span.find(
                        "span", class_=re.compile("\.*--color_black_100--\.*")
                    ).text
                    data[name] = value
            except:
                pass
            try:
                info_house = soup.find_all(
                    "div", class_=("a10a3f92e9--item--qJhdR"))
                for span in info_house:
                    name = span.find(
                        "span", class_=re.compile("\.*--color_gray60_100--\.*")
                    ).text
                    value = span.find(
                        "span", class_=re.compile("\.*--color_black_100--\.*")
                    ).text
                    data[name] = value
            except:
                pass
            lst.append(data)
    df = pd.DataFrame(lst)
    df.to_csv("data\cian.csv", index=False)
    return df

In [218]:
get_page_data(pages).head()

,Название,Адрес,Метро,Время до метро,Цена,Цена за квадрат,Общая площадь,Жилая площадь,Площадь кухни,Этаж,...,Тип перекрытий,Парковка,Подъезды,Отопление,Аварийность,Газоснабжение,Балкон/лоджия,Год сдачи,Дом,Отделка
0,"5-комн. квартира, 140,6 м²","Москва, ЦАО, р-н Тверской, ул. Петровка, 20/1",Трубная,5 мин.,82000000,583215,"140,6","81,6","15,6",5 из 5,...,Нет информации,Наземная,6,Центральное,Нет,Центральное,NaN,NaN,NaN,NaN
1,"Многокомнатная квартира, 190 м²","Москва, ЦАО, р-н Тверской, ул. Большая Дмитров...",Чеховская,7 мин.,89000000,468421,190,126,22,3 из 5,...,Нет информации,Наземная,2,Центральное,Нет,Центральное,NaN,NaN,NaN,NaN
2,"Многокомнатная квартира, 950 м²","Москва, ЦАО, р-н Пресненский, Гранатный пер., 8С4",Баррикадная,9 мин.,3488667000,3672281,950,NaN,NaN,8 из 8,...,Смешанные,Подземная,2,Индивидуальный тепловой пункт,Нет,NaN,"1 лоджия, 1 балкон",NaN,NaN,NaN
3,"5-комн. квартира, 200 м²","Москва, ЦАО, р-н Тверской, Тверская ул., 12С7",Пушкинская,3 мин.,75000000,375000,200,120,20,6 из 6,...,Нет информации,Наземная,5,Центральное,Нет,Центральное,NaN,NaN,NaN,NaN
4,"4-комн. апартаменты, 207,9 м²","Москва, ЦАО, р-н Тверской, ул. Охотный Ряд, 2",Охотный ряд,2 мин.,454889566,2188021,"207,9",NaN,NaN,5 из 15,...,Нет информации,NaN,NaN,Нет информации,Нет,NaN,NaN,NaN,NaN,NaN
